## Sourcing weather data with API and mergeing multiple datasets


In [1]:
import pandas as pd
import numpy as np
import os
import requests
import json
from datetime import datetime

### Import libraries & join bike trips data

In [19]:
# Create a list with all files in the folder using a list comprehension

folderpath = r"C:\Users\nasim\Citi-Bike\2022_citibike_data" 

# Join many CSV files
filepaths = [os.path.join(folderpath, name) for name in os.listdir(folderpath)]

In [ ]:
print(filepaths)

In [20]:
# Read and Join the Data Sets

df = pd.concat((pd.read_csv(f, low_memory=False) for f in filepaths), ignore_index=True)

In [ ]:
df.head()

In [ ]:
df.tail()

The most effective way to import and join multiple CSV files, such as these datasets, is to use a combination of a generator expression and pd.concat because:

1. Using a list of file paths allows you to store all CSV paths in a list (or generate them dynamically with os.listdir()), so the code can loop over them automatically.

2. Reading and concatenating with a generator expression ensures that each CSV is read only once and streamed directly into pd.concat.

Additionally, setting low_memory=False ensures that pandas correctly detects column types without producing warnings.

### Wrangling Citi-Bike dataset

In [23]:
# Covert the time frame of df dataset to year-month-day-time

df['started_at'] = pd.to_datetime(df['started_at'], format='%Y-%m-%d %H:%M:%S.%f')

In [24]:
# Extract only the date part (YYYY-MM-DD) from 'started_at' into a new column

df['date'] = pd.to_datetime(df['started_at'], format='%Y-%m-%d').dt.date

In [25]:
# Convert 'started_date' from string to datetime

df['date'] = pd.to_datetime(df['date'])

In [ ]:
df.head()

### Gathering Weather Data

In [30]:
# Define your NOAA token

Token = 'MVkdMZortXjCoeUBsoWVIQMTUBpSEfHz'

In [31]:
# Compile URL

r = requests.get('https://www.ncdc.noaa.gov/cdo-web/api/v2/data?datasetid=GHCND&datatypeid=TAVG&limit=1000&stationid=GHCND:USW00014732&startdate=2022-01-01&enddate=2022-12-31', headers={'token':Token})

In [32]:
# Load the api response as a json

d = json.loads(r.text)  

### Wrangling Weather dataset

In [33]:
# Secure all items in the response that correspond to TAVG

avg_temps = [item for item in d['results'] if item['datatype']=='TAVG']

In [34]:
# Get only the date field from all average temperature readings

data_temp = [item['date'] for item in avg_temps]

In [35]:
# Get the temperature from all average temperature readings

temps = [item['value'] for item in avg_temps]

In [36]:
# Create an empty DataFrame
df_temp = pd.DataFrame()

In [37]:
# Convert the dates to datetime (removes the time component)

df_temp['date'] = [datetime.strptime(d, "%Y-%m-%dT%H:%M:%S") for d in data_temp]

In [38]:
# Convert temperature from tenths of celsius to normal celsius

df_temp['avgTemp'] = [float(v)/10.0 for v in temps]

In [39]:
df_temp.tail()

,date,avgTemp
360,2022-12-27,-0.7
361,2022-12-28,3.4
362,2022-12-29,6.4
363,2022-12-30,9.3
364,2022-12-31,8.2


In [40]:
df_temp.head()

,date,avgTemp
0,2022-01-01,11.6
1,2022-01-02,11.4
2,2022-01-03,1.4
3,2022-01-04,-2.7
4,2022-01-05,3.2


### Merging two datasets

In [41]:
# Merge dataframes

df_merged = df.merge(df_temp, how = 'left', on = 'date', indicator = True)


In [ ]:
# Preview

df_merged.head()

In [43]:
# Checking the merge flag shows a 100 percent match rate between the two dataframes

df_merged['_merge'].value_counts(dropna = False)

_merge
both          29838166
left_only          640
right_only           0
Name: count, dtype: int64

There are 640 observations with NA which indicates there is something wrong in the merged datset.

In [ ]:
# Get the 640 rows that didn't merge

missing_merged = df_merged[df_merged['_merge'] == 'left_only']
# Preview them
missing_merged.tail()

The column "avgTemp" contains all 640 missing values. That is because of the recordings in the column 'started_at'. While the data downloaded from Kaggle is for 2022, it seems this column has some records for 2021 which are for the late time of the last night of 2021 such as 12/31/2021  11:56:05 PM started at and ended at 1/1/2022  12:10:04 AM. 

I decided to filter out these 640 recordings so that the starting date to be first day of 2022.

In [48]:
# Keep only rows that matched ('both') after merge

df_merged1 = df_merged[df_merged['_merge'] == 'both'].copy()

In [51]:
# Check years again

print(df_merged1['date'].dt.year.value_counts())

date
2022    29838166
Name: count, dtype: int64


In [52]:
# re-check the merged dataframe

df_merged1['_merge'].value_counts(dropna = False)

_merge
both          29838166
left_only            0
right_only           0
Name: count, dtype: int64

In [53]:
# Save the re-merged datset

df_merged1.to_csv('Merged_Bike_2022_data.csv')